### Install Dependencies

In [ ]:
!pip install nltk

### Imports

In [5]:
import nltk
import os
import zipfile
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

### Download NLTK Packages

In [ ]:
if not os.path.exists('./packages'):
    os.makedirs('./packages')
nltk.data.path.append('./packages')
nltk.download('all', download_dir='./packages')
with zipfile.ZipFile('./packages/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('./packages/corpora')

## Import Data

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./../../data/intents.json').read()
intents = json.loads(data_file)

## Check Data

In [4]:
intents.keys()

dict_keys(['intents'])

In [5]:
intents['intents'][:5]

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'context_set': ''},
 {'tag': 'creator',
  'patterns': ['what is the name of your developers',
   'what is the name of your creators',
   'what is the name of the developers',
   'what is the name of the creators',
   'who created you',
   'your developers',
   'your creators',
   'who are your developers',
   'developers',
   'you are made by',
   'you are made by whom',
   'who

##  Data pre-processing

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        # add to classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Lemmatization

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)


405 documents
38 classes ['admission', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'sem', 'sports', 'swear', 'syllabus', 'task', 'uniform', 'vacation']
263 unique lemmatized words ["'s", '(', ')', 'a', 'about', 'ac', 'active', 'activity', 'address', 'admision', 'admission', 'against', 'ai/ml', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'asshole', 'at', 'attend', 'automobile', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'boy', 'branch', 'bring', 'building', 'by', 'bye', 'cafetaria', 'call', 'called', 'campus', 'can', 'canteen', 'capacity', 'case', 'casuals', 'ce', 'chatting', 'chemical', 'civil', 'code', 'college', 'come', 'committe', 'comm

a pickle file is made that will be used for prediction.

In [8]:
pickle.dump(words,open('./bin/words.pkl','wb'))
pickle.dump(classes,open('./bin/classes.pkl','wb'))

# Create training and testing data
Now that the training data has been created, the input and output will be included. The pattern will serve as our input, and the class to which the pattern belongs will serve as our output. However, since the machine cannot interpret language, we must convert it to numbers.

In [9]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:,0]), dtype=np.float32)
train_y = np.array(list(training[:,1]), dtype=np.float32)
print("Training data created")

Training data created


# Build the model

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
81/81 [==============================] - 1s 2ms/step - loss: 3.6075 - accuracy: 0.0395
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4209 - accuracy: 0.1309
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2103 - accuracy: 0.1926
Epoch 4/200
81/81 [==============================] - 0s 3ms/step - loss: 2.9198 - accuracy: 0.2247
Epoch 5/200
81/81 [==============================] - 0s 3ms/step - loss: 2.6930 - accuracy: 0.2938
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.4807 - accuracy: 0.3407
Epoch 7/200
81/81 [==============================] - 0s 3ms/step - loss: 2.2439 - accuracy: 0.3901
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.0214 - accuracy: 0.4593
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.9467 - accuracy: 0.4667
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.7577 - accuracy: 0.5407
Epoch 11/

## Compile & Save

In [11]:
model.save('./bin/model.h5', hist)
print("model created")

## Test

In [6]:
model = load_model('./bin/model.h5')
intents = json.loads(open('./../../data/intents.json').read())
words = pickle.load(open('./bin/words.pkl','rb'))
classes = pickle.load(open('./bin/classes.pkl','rb'))

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag(sentence, words, show_details=True):
    sentence_words = clean_sentence(sentence)

    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def classify(sentence, model):
    p = bag(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']

    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = classify(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
chatbot_response('hello')

1/1 [==============================] - 3s 3s/step


'Good to see you again!'